In [3]:
import pandas as pd
import numpy as np
import simpy
import random

In [19]:
class Banco:
    def __init__(self, env, num_atms, num_cajas, num_agentes):
        self.env = env
        self.atms = simpy.Resource(env, capacity=num_atms)
        self.cajas = simpy.Resource(env, capacity=num_cajas)
        self.servicio_cliente = simpy.Resource(env, capacity=num_agentes)


In [ ]:
class Cliente:
    def __init__(self, env, id, banco: Banco):
        self.env = env
        self.id = id
        self.banco = banco


        self.tipo_servicio = random.choice(["atm", "caja", "servicio"])

        if self.tipo_servicio == "atm":
            self.operacion = random.choice(["retirar dinero", "consultar saldo"])
        elif self.tipo_servicio == "caja":
            self.operacion = random.choice(["retirar dinero", "depositar dinero", "cambiar billetes", "consultar saldo"])
        else:
            self.operacion = random.choice(["abrir cuenta", "solicitar préstamo", "cambiar datos personales", "reportar tarjeta"])

        self.eventos = pd.DataFrame()
        self.env.process(self.start())

    def start(self):
        llegada = self.env.now
        print(f"{llegada:.2f}: Cliente {self.id} llegó - quiere {self.tipo_servicio} para '{self.operacion}'")

        self.eventos = pd.concat([self.eventos, pd.DataFrame({
            "id": [self.id],
            "evento": ["LLEGO"],
            "time": [llegada],
            "servicio": [self.tipo_servicio],
            "operacion": [self.operacion]
        })])

      
        if self.tipo_servicio == "atm":
            recurso = self.banco.atms
            t_min, t_max = 1, 3
        elif self.tipo_servicio == "caja":
            recurso = self.banco.cajas
            t_min, t_max = 3, 6
        else:
            recurso = self.banco.servicio_cliente
            t_min, t_max = 6, 10

        with recurso.request() as req:
            yield req
            print(f"{self.env.now:.2f}: Cliente {self.id} siendo atendido ({self.tipo_servicio}) - '{self.operacion}'")

            self.eventos = pd.concat([self.eventos, pd.DataFrame({
                "id": [self.id],
                "evento": ["ATENDIDO"],
                "time": [self.env.now],
                "servicio": [self.tipo_servicio],
                "operacion": [self.operacion]
            })])

            duracion = random.randint(t_min, t_max)
            yield self.env.timeout(duracion)

            print(f"{self.env.now:.2f}: Cliente {self.id} finalizó ({self.tipo_servicio}) - '{self.operacion}'")
            self.eventos = pd.concat([self.eventos, pd.DataFrame({
                "id": [self.id],
                "evento": ["FINALIZADO"],
                "time": [self.env.now],
                "servicio": [self.tipo_servicio],
                "operacion": [self.operacion]
            })])


In [26]:
class Simulacion:
    def __init__(self, env, num_clientes, num_atms, num_cajas, num_agentes):
        self.env = env
        self.banco = Banco(env, num_atms, num_cajas, num_agentes)
        self.clientes = []

        for i in range(num_clientes):
            llegada = random.expovariate(1 / 9)  
            env.process(self.lanzar_cliente(i, llegada))

    def lanzar_cliente(self, id, delay):
        yield self.env.timeout(delay)
        cliente = Cliente(self.env, id, self.banco)
        self.clientes.append(cliente)


In [27]:
env = simpy.Environment()
random.seed(111)

num_clientes = 30
num_atms = 2
num_cajas = 4
num_agentes = 3

sim = Simulacion(env, num_clientes, num_atms, num_cajas, num_agentes)
env.run(until=60 * 8)


0.94: Cliente 27 llegó - quiere atm para 'retirar dinero'
0.94: Cliente 27 siendo atendido (atm) - 'retirar dinero'
1.03: Cliente 26 llegó - quiere atm para 'consultar saldo'
1.03: Cliente 26 siendo atendido (atm) - 'consultar saldo'
1.67: Cliente 6 llegó - quiere servicio para 'cambiar datos personales'
1.67: Cliente 6 siendo atendido (servicio) - 'cambiar datos personales'
1.94: Cliente 7 llegó - quiere caja para 'consultar saldo'
1.94: Cliente 7 siendo atendido (caja) - 'consultar saldo'
2.15: Cliente 1 llegó - quiere servicio para 'solicitar préstamo'
2.15: Cliente 1 siendo atendido (servicio) - 'solicitar préstamo'
2.18: Cliente 29 llegó - quiere atm para 'consultar saldo'
2.25: Cliente 14 llegó - quiere servicio para 'reportar tarjeta'
2.25: Cliente 14 siendo atendido (servicio) - 'reportar tarjeta'
2.35: Cliente 9 llegó - quiere servicio para 'abrir cuenta'
2.46: Cliente 21 llegó - quiere caja para 'depositar dinero'
2.46: Cliente 21 siendo atendido (caja) - 'depositar dinero'
2